In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
train = pd.read_csv("./data/train.csv", sep="###__###",header = None,encoding='utf-8')
# test = pd.read_csv("./data/test.csv", sep="###__###",header = None,encoding='utf-8')

In [2]:
train.columns = ['ID','Age','Gender','Education','query']
train.head()

,ID,Age,Gender,Education,query
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [3]:
train = train.query('Age > 0 and Gender > 0 and Education > 0')

In [4]:
import jieba
def extra(text):
    tokenized_words = []
    for query in text.split('\t'):
        tokenized_query = [w if w != ' ' else '<BLANK>'
                               for w in jieba.lcut(query)]
#         tokenized_query += ['<EOS>']
        tokenized_words.extend(tokenized_query)
    return' '.join(tokenized_words)

In [5]:
def get_search_num(text):
    return len(text.split('\t'))
train['search_num'] = train['query'].map(lambda x:get_search_num(x))
train['text_len'] = train['query'].map(lambda x:len(x.replace('\t','')))
train['text_avg_len'] = train['text_len']/train['search_num']

In [6]:
train['query'] = train['query'].map(lambda x:extra(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LZP\AppData\Local\Temp\jieba.cache
Loading model cost 0.681 seconds.
Prefix dict has been built successfully.


In [7]:
train.reset_index(drop=True, inplace=True)

In [12]:
train.to_csv('train.csv',index=None)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),#(1,3)
            min_df=3,  # 4  5
            max_df=0.9, # 0.95 1.0 
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)
train_term_doc = word_vec.fit_transform(train['query'])

# 逻辑回归

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=666)
label_arr = {'Gender':2,'Age':6,'Education':6}
for label in label_arr.keys():
    for i,(train_index,eval_index) in enumerate(kf.split(train_term_doc)):
        print(len(train_index),len(eval_index))

        lb = LabelEncoder()
        train[label] = lb.fit_transform(train[label].tolist())
        #训练集
        X_train = train_term_doc[train_index]
        y_train = train[label][train_index]

        #验证集
        X_eval = train_term_doc[eval_index]
        y_eval = train[label][eval_index]

        model = LogisticRegression(C=4, dual=True) 
        model.fit(X_train,y_train)

        ####对于验证集进行预测
        train_matrix = np.zeros((train.shape[0],label_arr[label]))
        eval_prob = model.predict_proba(X_eval)
        train_matrix[eval_index] = eval_prob.reshape((X_eval.shape[0], label_arr[label]))#array

        eval_pred = np.argmax(eval_prob,axis=1)
        eval_pred = lb.inverse_transform(eval_pred)
        score = metrics.accuracy_score(lb.inverse_transform(y_eval),eval_pred)
        
        ###保存模型和数据
        
#         model.save
#         cv_scores.append(score)
        print("-----{}:第{}次-----".format(label,i))
        print("validation score is",score)

70793 17699
-----Age:第0次-----
validation score is 0.61325498615741
70793 17699
-----Age:第1次-----
validation score is 0.6172100118650771
70794 17698
-----Age:第2次-----
validation score is 0.6197310430557125
70794 17698
-----Age:第3次-----
validation score is 0.6148152333597017
70794 17698
-----Age:第4次-----
validation score is 0.6104644592609334
70793 17699
-----Gender:第0次-----
validation score is 0.8441719871179163
70793 17699
-----Gender:第1次-----
validation score is 0.8476185095203119
70794 17698
-----Gender:第2次-----
validation score is 0.8407729686970279
70794 17698
-----Gender:第3次-----
validation score is 0.8442196858402079
70794 17698
-----Gender:第4次-----
validation score is 0.8431461182054469
70793 17699
-----Education:第0次-----
validation score is 0.6390756539917509
70793 17699
-----Education:第1次-----
validation score is 0.6376631448104413
70794 17698
-----Education:第2次-----
validation score is 0.6380381964063736
70794 17698
-----Education:第3次-----
validation score is 0.63413945078539

# LGBM

In [14]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import numpy as np
import lightgbm as lgb
#sklearn api
kf = KFold(n_splits=5, shuffle=True, random_state=666)
label_arr = {'Gender':2,'Age':6,'Education':6}
# label_arr = {'Age':6,'Gender':2,'Education':6}
lgbm_class = {'Age':'multiclass','Gender':'binary','Education':'multiclass'}
lgbm_metric = {'Age':'multi_logloss','Gender':'binary_logloss','Education':'multi_logloss'}
for label in label_arr.keys():
    for i,(train_index,eval_index) in enumerate(kf.split(train_term_doc)):
        print(len(train_index),len(eval_index))

        lb = LabelEncoder()
        train[label] = lb.fit_transform(train[label].tolist())
        #训练集
        X_train = train_term_doc[train_index]
        y_train = train[label][train_index]

        #验证集
        X_eval = train_term_doc[eval_index]
        y_eval = train[label][eval_index]

        model =lgb.LGBMClassifier(boosting_type='gbdt', 
                   num_leaves=2**5,
                   max_depth=-1, 
                   learning_rate= 0.1,
                   n_estimators=10,
                   objective=lgbm_class[label],
                   subsample=0.7,#
                   colsample_bytree=0.5,#
                   reg_lambda=10,#l2
                   n_jobs=16, #
#                    num_class=label_arr[label],#
                   silent=True, 
                   random_state=2019,
#                    class_weight=20,
                   colsample_bylevel=0.5,
                   min_child_weight=1.5,
                   metric=lgbm_metric[label]
                  )
        model.fit(X_train,y_train,eval_set=(X_eval,y_eval), early_stopping_rounds=2)

        ####对于验证集进行预测
        train_matrix = np.zeros((train.shape[0],label_arr[label]))
        eval_prob = model.predict_proba(X_eval)
#         print(eval_prob.tolist())
        print(eval_prob.shape)
#         train_matrix[eval_index] = eval_prob..reshape((X_eval.shape[0], label_arr[label]))#array
#         if label == 'Gender':
#             eval_prob = eval_prob[0].reshape((label_arr[label],X_eval.shape[0])).T
        eval_pred = np.argmax(eval_prob,axis=1)
        eval_pred = lb.inverse_transform(eval_pred)
        score = metrics.accuracy_score(lb.inverse_transform(y_eval),eval_pred)
        
        ###保存模型和数据
        
#         model.save
#         cv_scores.append(score)
        print("-----{}:第{}次-----".format(label,i))
        print("validation score is",score)

70793 17699
[1]	valid_0's binary_logloss: 0.66014
Training until validation scores don't improve for 2 rounds
[2]	valid_0's binary_logloss: 0.642805
[3]	valid_0's binary_logloss: 0.629215
[4]	valid_0's binary_logloss: 0.616363
[5]	valid_0's binary_logloss: 0.605512
[6]	valid_0's binary_logloss: 0.596151
[7]	valid_0's binary_logloss: 0.586473
[8]	valid_0's binary_logloss: 0.5775
[9]	valid_0's binary_logloss: 0.570164
[10]	valid_0's binary_logloss: 0.564121
Did not meet early stopping. Best iteration is:
[10]	valid_0's binary_logloss: 0.564121
(17699, 2)
-----Gender:第0次-----
validation score is 0.7427538278998813
70793 17699


KeyboardInterrupt: 